In [146]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import SystemMessage, HumanMessage
import operator
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

In [147]:
load_dotenv()

True

In [148]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation")
model = ChatHuggingFace(llm=llm)

In [149]:
class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(..., description="The evaluation of the tweet")
    feedback: str = Field(..., description="Feedback on how to improve the tweet if needed")

In [150]:
parser = PydanticOutputParser(pydantic_object=TweetEvaluation)

In [151]:
prompt = PromptTemplate(
    template=(
        "Analyze the following tweet and evaluate it based on clarity, engagement, and appropriateness.\n\n"
        "Tweet: {tweet}\n\n"
        "Return your answer strictly in the following JSON format:\n{response_format}"
    ),
    input_variables=['tweet'],
    partial_variables={'response_format': parser.get_format_instructions()}
)


In [152]:
structured_evaluator_llm = prompt | model | parser

In [153]:
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal['approved', 'needs_improvement']
    feedback: str
    iteration: int
    max_iteration: int

In [154]:
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]
    response = model.invoke(messages).content
    new_state = state.copy()
    new_state['tweet'] = response
    return new_state

In [155]:
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
        HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
    ]
    response = structured_evaluator_llm.invoke(messages)
    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [156]:
def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]
    response = model.invoke(messages).content
    new_state = state.copy()
    new_state['tweet'] = response
    new_state['iteration'] = state['iteration'] + 1
    return new_state


In [158]:
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'
    

In [159]:
graph = StateGraph(TweetState)
graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)


graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')
workflow = graph.compile()


In [162]:
initial_state = {
    'topic': "The future of AI in everyday life",
    'iteration': 1,
    'max_iteration': 5
}
final_state = workflow.invoke(initial_state)
print(final_state)

{'topic': 'The future of AI in everyday life', 'tweet': "\nThere's a reason why the future hasn't arrived yet - it's been tied to your shoelaces every morning. But when it does, expect AI to roll over, hit the snooze button, and beg for more coffee, just like the rest of us. ☕️🤖💤\n\n", 'evaluation': 'approved', 'feedback': 'This tweet is original and nicely formatted with the literary device of personification. It is punchy, and the humor is evident. The content is entertaining and thought-provoking without exceeding the character limit of 280 characters. The tweet is not a question-answer format or a setup-punchline joke, and it does not end with generic, deflating, or throwaway lines. It is a well-structured tweet that has virality potential.', 'iteration': 1, 'max_iteration': 5}
